# 시맨틱 검색 엔진 구축하기
이 노트북은 LangChain의 문서 로더, 임베딩, 그리고 벡터 스토어 추상화 개념에 익숙해지도록 설계되었습니다. 이러한 추상화는 (벡터) 데이터베이스 및 기타 소스에서 데이터를 검색하여 LLM 워크플로우에 통합하도록 지원합니다. 이는 **RAG (Retrieval-Augmented Generation)** 와 같은 모델 추론의 일부로 데이터를 검색해 활용하는 애플리케이션에 중요합니다.

이 노트북에서는 PDF 문서에 대한 검색 엔진을 구축합니다. 이를 통해 입력 쿼리와 유사한 PDF의 특정 구절을 검색할 수 있습니다.

### 개념
이 가이드는 텍스트 데이터 검색에 중점을 둡니다. 아래 개념을 다룰 것입니다:

- 문서 및 문서 로더
- 텍스트 분할기 (Text splitters)
- 임베딩 (Embeddings)
- 벡터 스토어 및 검색기 (Vector stores and retrievers)

### 문서 및 문서 로더 (Documents and Document Loaders)
LangChain은 Document 추상화를 제공합니다. 이 객체는 텍스트 단위와 관련 메타데이터를 나타내기 위해 설계되었습니다.  

Document의 주요 속성:  
- page_content: 문서의 내용을 나타내는 문자열입니다.  
- metadata: 문서의 출처, 다른 문서와의 관계 등 임의의 메타데이터를 포함하는 딕셔너리입니다.  
- id: (선택적) 문서를 식별하는 문자열입니다.  

metadata 속성은 문서의 출처, 문서 간 관계 및 기타 정보를 포함할 수 있습니다. 일반적으로 하나의 Document 객체는 더 큰 문서의 일부(청크)를 나타냅니다.

예제 문서 생성:
```python
from langchain_core.documents import Document

documents = [
    Document(
        page_content="개는 충성심과 친근함으로 잘 알려진 훌륭한 동반자입니다. ",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="고양이는 독립적인 반려동물로, 종종 자신만의 공간을 즐깁니다.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
```

그러나 LangChain 생태계는 문서 로더(document loaders)를 구현하여 수백 가지 일반적인 소스와 통합할 수 있습니다. 이를 통해 이러한 소스의 데이터를 AI 애플리케이션에 쉽게 통합할 수 있습니다.


In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv \
# pypdf

## **문서 로드하기**

PDF 파일을 `Document` 객체의 시퀀스로 로드해 보겠습니다. LangChain 저장소에 [예제 PDF](https://github.com/langchain-ai/langchain/tree/master/docs/docs/example_data)가 있습니다. 이 PDF는 **2023년 나이키(Nike)의 10-K 보고서**입니다.  

**`PyPDFLoader`** 는 **PDF의 각 페이지마다 하나의 `Document` 객체**를 로드합니다.  

각 객체에서 다음 정보를 쉽게 접근할 수 있습니다:  

- **페이지의 문자열 콘텐츠**  
- **파일 이름과 페이지 번호를 포함한 메타데이터**  

## **텍스트 분할 (Splitting)**  

정보 검색 및 후속 질문-응답(question-answering) 작업을 하는데 **페이지 단위** 사용은 너무 거친 표현일 수 있습니다. 우리의 최종 목표는 **입력 쿼리에 답변할 수 있는 `Document` 객체를 검색하는 것**이므로, PDF를 더 세분화하면 문서의 해당되는 부분이 주변 텍스트에 의해 의미가 흐려지는 것을 방지할 수 있습니다.  

이를 위해 텍스트 분할기(text splitters)를 사용할 수 있습니다. 여기서는 **문자를 기준으로 분할하는 간단한 텍스트 분할기**를 사용할 것입니다.  

- 문서를 **1000자 단위로 분할**합니다.  
- 각 청크(chunk) 사이에는 **200자의 중첩(overlap)** 이 있습니다.  

**중첩**은 중요한 문맥(context)과 문장(statement)이 분리되는 것을 완화하는 데 도움이 됩니다.  

우리는 **RecursiveCharacterTextSplitter**를 사용할 것입니다. 이 분할기는 일반적인 구분자(예: 줄바꿈)를 사용하여 **재귀적으로 문서를 분할**하며, 각 청크가 적절한 크기가 될 때까지 반복합니다. **일반 텍스트 사용 사례에서 권장되는 텍스트 분할기입니다.**  

### **추가 설정**  
- `add_start_index=True`로 설정하면 **각 분할된 `Document`가 원본 `Document` 내에서 시작하는 문자 인덱스**가 **메타데이터 속성(`start_index`)** 으로 보존됩니다.  

In [ ]:
# RecursiveCharacterTextSplitter를 가져옵니다.
# 이 도구는 텍스트를 재귀적으로 분할하며, 일반적인 구분자(예: 줄바꿈)를 사용합니다.
# RecursiveCharacterTextSplitter 설정
# 문서를 분할합니다. 'docs'는 분할할 원본 문서 목록입니다.
# 분할된 문서의 총 개수

## **임베딩 (Embeddings)**  

**벡터 검색(Vector Search)** 은 비정형 데이터(예: 비정형 텍스트)를 저장하고 검색하는 일반적인 방법입니다. 핵심 아이디어는 텍스트와 연결된 **숫자 벡터(Numeric Vectors)** 를 저장하는 것입니다.  

쿼리(Query)가 주어지면, 이를 **동일한 차원의 벡터로 임베딩(Embedding)** 하고 **벡터 유사도 측정(metric)** (예: **코사인 유사도**)을 사용하여 관련 텍스트를 식별할 수 있습니다.   

### **모델 선택하기**  

원하는 임베딩 모델을 선택해 사용합니다.  
예를 들어, OpenAI, Hugging Face, Cohere 등 다양한 제공자를 사용할 수 있습니다.  

**임베딩 모델을 선택한 후 벡터 스토어(Vector Store)에 텍스트를 저장하고 검색**할 수 있습니다.

In [ ]:
# 쿼리 텍스트를 벡터로 임베딩합니다.
# 각 청크의 내용을 벡터로 변환하여 비교합니다.
# 첫 번째 청크(all_splits[0])의 내용을 벡터로 임베딩합니다.
# 두 번째 청크(all_splits[1])의 내용을 벡터로 임베딩합니다.
# 두 벡터의 길이가 동일한지 확인합니다.
# 벡터의 길이를 출력합니다.
# 첫 번째 벡터의 일부(처음 10개 요소)를 출력합니다.

텍스트 임베딩을 생성하는 모델을 준비했다면, 이제 이를 효율적인 **유사도 검색(Similarity Search)** 을 지원하는 **벡터 스토어(Vector Stores)** 에 저장할 수 있습니다.

---

## **벡터 스토어(Vector Stores)**  

LangChain의 [**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 객체는 **텍스트 및 `Document` 객체를 저장**하고, 다양한 **유사도 메트릭(Similarity Metrics)**을 사용해 쿼리를 수행할 수 있는 메서드를 포함합니다.  

이 객체들은 종종 **임베딩(Embedding)** 모델로 초기화되며, 해당 모델은 **텍스트 데이터를 숫자 벡터로 변환하는 방법**을 결정합니다.   

- 일부 벡터 스토어는 **클라우드 제공업체(Cloud Providers)** 에서 호스팅되며 특정 **자격 증명(Credentials)** 이 필요합니다.  
- 일부 벡터 스토어(예: **Postgres**)는 독립적인 인프라에서 실행되거나 로컬 또는 서드파티 플랫폼을 통해 운영될 수 있습니다.  
- 일부 벡터 스토어는 **인메모리(In-Memory)** 로 실행되어 가벼운 작업에 적합합니다.  

In [ ]:
# !pip install -q chromadb
# !pip install -q langchain-chroma

벡터 저장소를 인스턴스화했으므로 이제 문서를 인덱싱할 수 있습니다.

In [ ]:
# 문서 목록을 벡터 스토어에 추가하고, 각 문서에 대한 고유 식별자(ID) 반환
# - documents: 추가할 문서 목록 (여기서는 'all_splits'로 분할된 문서 목록)

## **VectorStore 쿼리하기**  

**`VectorStore`** 가 문서를 포함하고 있는 상태에서, 우리는 이를 쿼리할 수 있습니다.  
[**VectorStore**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html)는 다음과 같은 쿼리 메서드를 제공합니다:  

- **동기(Synchronous) 및 비동기(Asynchronous) 방식**  
- **문자열(String) 쿼리 또는 벡터(Vector) 기반 쿼리**  
- **유사도 점수(Similarity Scores) 반환 여부 선택**  
- **유사도(Similarity)** 및 [**최대 주변부 중요도(Maximum Marginal Relevance)**](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html#langchain_core.vectorstores.base.VectorStore.max_marginal_relevance_search)를 사용하여 검색된 결과의 다양성을 균형 있게 조정  

일반적으로 이 메서드들은 출력값으로 [**Document**](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 객체 목록을 포함합니다.  

---

**임베딩(Embeddings)** 은 일반적으로 텍스트를 **'밀집 벡터(Dense Vector)'** 로 표현합니다. 이로 인해 **의미가 유사한 텍스트는 기하학적으로 가까운 위치**에 있게 됩니다.  
이를 통해 **문서에 사용된 특정 키워드를 알지 못해도** 질문을 입력하는 것만으로 관련 정보를 검색할 수 있습니다.  

Return scores (점수 반환:):

In [ ]:
# 'similarity_search_with_score' 메서드는 쿼리와 가장 유사한 문서를 반환합니다.
# 반환값: (문서 객체, 유사도 점수) 쌍의 리스트
# 첫 번째 결과를 추출합니다.
# 유사도 점수 출력
# 검색된 문서 내용 출력

----------
내장된 쿼리와의 유사성을 기준으로 문서 반환:

## **검색기 (Retrievers)**  

LangChain의 **`VectorStore` 객체**는 [**Runnable**](https://python.langchain.com/api_reference/core/index.html#langchain-core-runnables)의 서브클래스가 아닙니다. 반면, **[Retrievers](https://python.langchain.com/api_reference/core/index.html#langchain-core-retrievers)** 는 **Runnable**의 서브클래스입니다. 따라서 **동기(Synchronous)** 및 **비동기(Asynchronous)** `invoke`와 `batch` 작업과 같은 표준 메서드 집합을 구현합니다.  

### **검색기 vs 벡터 스토어**  
- **벡터 스토어(VectorStore)**: 벡터화된 데이터를 저장하고 쿼리합니다.  
- **검색기(Retriever)**: 벡터 스토어를 활용하거나 외부 API와 같은 **비벡터 데이터 소스**와도 통신할 수 있습니다.  

---

## **검색기 직접 생성하기**  

`Retriever`를 서브클래싱하지 않고도 간단한 버전을 직접 생성할 수 있습니다.  
- 원하는 **문서 검색 메서드**를 선택합니다.  
- 해당 메서드를 사용해 **Runnable** 객체를 쉽게 생성할 수 있습니다.  

###  **예시: `similarity_search`를 활용한 검색기**  

다음 단계에서는 `similarity_search` 메서드를 기반으로 간단한 검색기를 구축해 보겠습니다.

In [ ]:
# 사용자 정의 검색기(Retriever) 생성
# 이 검색기는 주어진 쿼리(query)를 기반으로 벡터 스토어에서 문서를 검색합니다.
def retriever(query: str) -> List[Document]:
# 여러 개의 쿼리를 동시에 배치(batch) 검색 실행하여 검색 결과를 반환

**벡터 스토어(Vectorstores)** 는 **`as_retriever`** 메서드를 구현하며, 이를 통해 **[VectorStoreRetriever](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html)** 를 생성할 수 있습니다.  

이 검색기(Retriever)는 특정 **`search_type`** 및 **`search_kwargs`** 속성을 포함하며, 이를 통해 **벡터 스토어의 어떤 메서드를 호출할지**, 그리고 **어떻게 매개변수를 설정할지**를 정의합니다.  

예를 들어, 위의 예제를 다음과 같이 재현할 수 있습니다:  

In [ ]:
# `as_retriever` 메서드를 사용하여 벡터 스토어를 검색기 객체로 변환합니다.
# 여러 개의 쿼리를 동시에 실행하여 검색 결과를 반환합니다.